### 1. General info of dataset GSE136929

This is the Jupyter Notebook for dataset GSE136929. Its dataset includes barcodes/genes/matrix files for each sample. There are 2 samples.


<span style="color:green">**[D90-rROs]**</span> Normal human retinal organoids at 90-days

<span style="color:green">**[D90-rRBOs]**</span> Human Rb organoids (hRBOs) at 90-days

In [1]:
# Environment setup
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as anndata
import scipy

### 2. AnnData object of each sample

<span style="color:red">**IMPORTANT:**</span> rename files to get rid of prefixes

1. `barcodes.tsv`: cell barcodes, which go into `.obs`
2. `genes.tsv`: gene names, `.var`
3. `matrix.mtx`: the expression matrix, `.X`

In [21]:
from pathlib import Path

# Specify directory paths
data_directory = Path('/scratch/user/uqjsaxo1/xiaohan-john-project/data/GSE136929_RAW/')
write_directory = Path('/scratch/user/uqjsaxo1/xiaohan-john-project/write/GSE136929/')

# Loop through all files in the directory
for sample_directory in data_directory.iterdir():
    sample_name = sample_directory.stem
    sample_h5ad = sample_name + '_uni.h5ad'

    sample = sc.read_10x_mtx(
    sample_directory,
    var_names='gene_symbols',  
    cache=False
    )

    # Create an observation metric info to store related features
    obs_metrics = pd.DataFrame(index=sample.obs_names) ## Get the identifiers
    obs_metrics['cancer_type'] = 'Retinoblastoma'
    obs_metrics['dataset'] = 'GSE136929'
    obs_metrics['tissue'] = 'retinal organoid'
    obs_metrics['sample_name'] = sample_name
    obs_metrics['uni_barcode'] = obs_metrics['dataset'] + '_' + obs_metrics.index.astype(str)
    
    sample.obs = obs_metrics
    sample.obs.set_index("uni_barcode", drop=True, inplace=True)
    print(sample)

    # save the anndata object
    output_path = write_directory.joinpath(sample_h5ad)
    sample.write_h5ad(output_path, compression="gzip")

AnnData object with n_obs × n_vars = 9665 × 19997
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_name'
    var: 'gene_ids'
AnnData object with n_obs × n_vars = 12218 × 19997
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_name'
    var: 'gene_ids'


### 3. Confirmation of created AnnData objects

In [22]:
from pathlib import Path

# Specify directory paths
write_directory = Path('/scratch/user/uqjsaxo1/xiaohan-john-project/write/GSE136929/')

# Loop through all files in the directory
for file in write_directory.iterdir():
    sample = anndata.read_h5ad(file)
    print(sample)

AnnData object with n_obs × n_vars = 12218 × 19997
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_name'
    var: 'gene_ids'
AnnData object with n_obs × n_vars = 9665 × 19997
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_name'
    var: 'gene_ids'


In [23]:
sample.obs

,cancer_type,dataset,tissue,sample_name
uni_barcode,,,,
GSE136929_AAACCTGAGCCCTAAT-1,Retinoblastoma,GSE136929,retinal organoid,GSM5468186_D90-hRBOs
GSE136929_AAACCTGAGCCGTCGT-1,Retinoblastoma,GSE136929,retinal organoid,GSM5468186_D90-hRBOs
GSE136929_AAACCTGAGTCCGTAT-1,Retinoblastoma,GSE136929,retinal organoid,GSM5468186_D90-hRBOs
GSE136929_AAACCTGCAATCCGAT-1,Retinoblastoma,GSE136929,retinal organoid,GSM5468186_D90-hRBOs
GSE136929_AAACCTGCACAGACAG-1,Retinoblastoma,GSE136929,retinal organoid,GSM5468186_D90-hRBOs
...,...,...,...,...
GSE136929_TTTGTCAGTTCAACCA-1,Retinoblastoma,GSE136929,retinal organoid,GSM5468186_D90-hRBOs
GSE136929_TTTGTCATCATAAAGG-1,Retinoblastoma,GSE136929,retinal organoid,GSM5468186_D90-hRBOs
GSE136929_TTTGTCATCTCCTATA-1,Retinoblastoma,GSE136929,retinal organoid,GSM5468186_D90-hRBOs


In [20]:
sample.var

,gene_ids
OR4F5,ENSG00000186092
OR4F29,ENSG00000284733
OR4F16,ENSG00000284662
SAMD11,ENSG00000187634
NOC2L,ENSG00000188976
...,...
AC233755.2,ENSG00000277856
AC233755.1,ENSG00000275063
AC240274.1,ENSG00000271254
AC213203.2,ENSG00000277475


### 4. Convert AnnData objects to SingleCellExperiment objects

In [3]:
from pathlib import Path

import anndata2ri
import rpy2.robjects as robjects
from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE136929')

# Loop through all files in the directory
for file in write_directory.iterdir():
    sample_name = file.stem
    if "_uni.h5ad" in file.name:
        sample_anndata = anndata.read_h5ad(file)
        sample_sce_file = sample_name + ".rds"

        with localconverter(anndata2ri.converter):
            sample_sce = anndata2ri.py2rpy(sample_anndata)
        # print(sample_sce)
        
        # Save the sce object in .rds file
        robjects.globalenv["sample_sce"] = sample_sce
        sample_sce_path = write_directory / sample_sce_file
        robjects.r("saveRDS(sample_sce, file='{}')".format(sample_sce_path))

class: SingleCellExperiment 
dim: 19997 12218 
metadata(0):
assays(1): X
rownames(19997): OR4F5 OR4F29 ... AC213203.2 AC213203.1
rowData names(1): gene_ids
colnames(12218): GSE136929_AAACCTGAGCCACGTC-1
  GSE136929_AAACCTGAGCCTATGT-1 ... GSE136929_TTTGTCATCGGAGCAA-1
  GSE136929_TTTGTCATCTCCAACC-1
colData names(4): cancer_type dataset tissue sample_name
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

class: SingleCellExperiment 
dim: 19997 9665 
metadata(0):
assays(1): X
rownames(19997): OR4F5 OR4F29 ... AC213203.2 AC213203.1
rowData names(1): gene_ids
colnames(9665): GSE136929_AAACCTGAGCCCTAAT-1
  GSE136929_AAACCTGAGCCGTCGT-1 ... GSE136929_TTTGTCATCTGAAAGA-1
  GSE136929_TTTGTCATCTGCGTAA-1
colData names(4): cancer_type dataset tissue sample_name
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

